# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [1]:
!rm -rf TTS_HW
!git clone https://github.com/Howuhh/TTS_HW #####YOUR FORK#####
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz
# !curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
# !tar -xf libri_training_data.tar.gz

Cloning into 'TTS_HW'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 295 (delta 83), reused 272 (delta 66), pack-reused 0
Receiving objects: 100% (295/295), 4.88 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (83/83), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0  10413      0 --:--:-- --:--:-- --:--:-- 10413
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3287    0  3287    0     0  20936      0 --:--:-- --:--:-- --:--:-- 20936
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [2]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

In [3]:
!pip install -r TTS_HW/requirements.txt
# !pip install -r requirements.txt
!pip install transformers
!pip install torchmetrics

  Using cached DLLogger-0.1.0-py3-none-any.whl


In [4]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh
# !bash scripts/download_cmudict.sh
# !bash scripts/download_fastpitch.sh
# !bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [21]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-10 18:53:37.424013 - PARAMETER | output :  ./output
DLL 2021-12-10 18:53:37.424161 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-10 18:53:37.424237 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-10 18:53:37.424340 - PARAMETER | pyprof :  False
DLL 2021-12-10 18:53:37.424525 - PARAMETER | epochs :  5
DLL 2021-12-10 18:53:37.424745 - PARAMETER | epochs_per_che

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [25]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-10 19:02:07.613553 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-10 19:02:07.613617 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-10 19:02:07.613686 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-10 19:02:07.613787 - PARAMETER | save_mels :  False
DLL 2021-12-10 19:02:07.613855 - PARAMETER | cuda :  True
DLL 2021-12-10 19:02:07.613933 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-10 19:02:07.613995 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_5.pt
DLL 2021-12-10 19:02:07.614073 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-10 19:02:07.614141 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-10 19:02:07.614214 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-10 19:02:07.614276 - PARAMETER | sampling_rate :  22050
DLL 2021-12-10 19:02:07.614338 - PARAMETER | stft_hop_length :  256
DLL 2021-12-10 19:02:07.614401 - PAR

In [24]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [26]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [5]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, RobertaTokenizer, BertModel, RobertaForTokenClassification
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [8]:
def val_to_class(val):
    if type(val) == str:
        val = float(val)
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3).to(device)
acc_criterion = torchmetrics.Accuracy(num_classes=3).to(device)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        self.tokenized = self.tokenizer(
            sentences, 
            padding=True, 
            truncation=True, 
            max_length=self.max_len,
            return_tensors="pt"
        )
            
    def __getitem__(self, index):
        #####################
        ##### YOUR CODE #####
        #####################
        ids = self.tokenized.input_ids[index]
        mask = self.tokenized.attention_mask[index]
        
        # pad labels for padding tokens
        padded_max_len = self.tokenized.input_ids.shape[-1]
        
        labels = [val_to_class(v) for v in self.labels[index]]
        if len(labels) > padded_max_len:
            labels = labels[:padded_max_len]
        else:
            labels.extend(
                [0] * (padded_max_len - len(labels))
            )
        # labels = [0] + labels # CLS token

        return {
            'input_ids': ids,
            'attention_mask': mask,
            'labels': torch.tensor(labels, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    #####################
    ##### YOUR CODE #####
    #####################
    with open(filename, "r") as f:
        sentence = []
        labels = []
        for line in f:
            line = line.split("\t")
            sentence.append(line[-3])
            
            # num_tokens = len(tokenizer(line[-3], add_special_tokens=False).input_ids)
            # labels.extend([float(line[-2])] * num_tokens)
            labels.append(float(line[-2]))

            
        sentences.append(" ".join(sentence))
        label_proms.append(labels)

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-20], label_proms[:-20], 100)
test_dataset = PromDataset(tokenizer, sentences[-20:], label_proms[-20:], 100)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

##### YOUR CODE #####
model = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        #####################
        ##### YOUR CODE #####
        #####################
        batch = {k: v.to(device) for k, v in batch.items()}
        
        out = model(**batch)
        loss = out.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t_bar.set_description(desc=f'loss; {loss.item()}')
        
    
    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            #####################
            ##### YOUR CODE #####
            #####################
            batch = {k: v.to(device) for k, v in batch.items()}
        
            out = model(**batch)
            mask = batch["attention_mask"]
            
            pred = out.logits.argmax(-1)
            pred = pred.masked_select(mask == 1)
            target = batch["labels"].masked_select(mask == 1)
            
            accs.append(acc_criterion(pred, target))
            f1_scores.append(f1_criterion(pred, target))

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

EPOCH:0


loss; 0.6608129739761353: 100%|██████████| 14/14 [00:08<00:00,  1.60it/s]


Accuracy: 0.791743814945221
F1 score: 0.7917437553405762
EPOCH:1


loss; 0.4922263026237488: 100%|██████████| 14/14 [00:08<00:00,  1.62it/s]


...

Accuracy: 0.7735745906829834
F1 score: 0.7735745906829834
EPOCH:8


loss; 0.4161464273929596: 100%|██████████| 14/14 [00:08<00:00,  1.58it/s]


Accuracy: 0.755842924118042
F1 score: 0.755842924118042
EPOCH:9


loss; 0.45970582962036133: 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]

Accuracy: 0.7635135054588318
F1 score: 0.7635135054588318


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!